<a href="https://colab.research.google.com/github/CharmStrange/CoLab_data/blob/main/ipynbs/D.E./AirFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%pip install "apache-airflow[celery]==2.7.1" --constraint "https://raw.githubusercontent.com/apache/airflow/constraints-2.7.1/constraints-3.8.txt"
%pip install "apache-airflow==2.7.1" apache-airflow-providers-google==10.1.0

  Using cached apache_airflow-2.7.1-py3-none-any.whl (12.9 MB)
  Using cached alembic-1.12.0-py3-none-any.whl (226 kB)
  Using cached argcomplete-3.1.1-py3-none-any.whl (41 kB)
  Using cached asgiref-3.7.2-py3-none-any.whl (24 kB)
  Using cached blinker-1.6.2-py3-none-any.whl (13 kB)
  Using cached cattrs-23.1.2-py3-none-any.whl (50 kB)
  Using cached connexion-2.14.2-py2.py3-none-any.whl (95 kB)
  Using cached croniter-1.4.1-py2.py3-none-any.whl (19 kB)
  Using cached cryptography-41.0.3-cp37-abi3-manylinux_2_28_x86_64.whl (4.3 MB)
  Using cached Flask_AppBuilder-4.3.6-py3-none-any.whl (1.7 MB)
  Using cached Flask_Caching-2.0.2-py3-none-any.whl (28 kB)
  Using cached Flask_Login-0.6.2-py3-none-any.whl (17 kB)
  Using cached flask_session-0.5.0-py3-none-any.whl (7.2 kB)
  Using cached Flask_WTF-1.1.1-py3-none-any.whl (12 kB)
  Using cached gunicorn-21.2.0-py3-none-any.whl (80 kB)
  Using cached httpx-0.23.3-py3-none-any.whl (71 kB)
  Using cached jsonschema-4.19.0-py3-none-any.whl (83

In [ ]:
import os

from datetime import datetime, timedelta

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator

DATA_FOLDER = "data"


def decorate_file(input_path, output_path):
    with open(input_path, "r") as in_file:
        line = in_file.read()

    with open(output_path, "w") as out_file:
        out_file.write("My "+line)


default_args = {
    "owner": "lorenzo",
    "depends_on_past": False,
    "start_date": datetime(2018, 9, 20),
    "email": ["l.peppoloni@gmail.com"],
    "email_on_failure": False,
    "email_on_retry": False,
    "retries": 0,
}


dag = DAG(
    "multiple_files_dag",
    default_args=default_args,
    schedule_interval="0 12 * * *",
    )

for i in range(10):
    output_one_path = os.path.join(DATA_FOLDER, "output_one_{:d}.txt".format(i))
    output_two_path = os.path.join(DATA_FOLDER, "output_two_{:d}.txt".format(i))

    t1 = BashOperator(
        task_id="print_file_{:d}".format(i),
        bash_command='echo "pipeline" > {}'.format(output_one_path),
        dag=dag)

    t2 = PythonOperator(
        task_id="decorate_file_{:d}".format(i),
        python_callable=decorate_file,
        op_kwargs={"input_path": output_one_path, "output_path": output_two_path},
        dag=dag)

    t2.set_upstream(t1)

<ipython-input-2-9a97bf721742>:6 DeprecationWarning: The `airflow.operators.bash_operator.BashOperator` class is deprecated. Please use `'airflow.operators.bash.BashOperator'`.

<ipython-input-2-9a97bf721742>:7 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-2-9a97bf721742>:31 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

DAG를 정의하고 초기화하고 DAG에서 사용될 두 연산자 `BashOperator`와 `PythonOperator` 추가.

---

여러 파일이 포함된 병렬 DAG AirFlow

In [ ]:
import os

from datetime import datetime, timedelta

from airflow import DAG
from airflow.operators.bash_operator import BashOperator
from airflow.operators.python_operator import PythonOperator

DATA_FOLDER = "data"


def decorate_file(input_path, output_path):
    with open(input_path, "r") as in_file:
        line = in_file.read()

    with open(output_path, "w") as out_file:
        out_file.write("My "+line)


default_args = {
    "owner": "lorenzo",
    "depends_on_past": False,
    "start_date": datetime(2018, 9, 20),
    "email": ["l.peppoloni@gmail.com"],
    "email_on_failure": False,
    "email_on_retry": False,
    "retries": 0,
}


dag = DAG(
    "multiple_files_dag",
    default_args=default_args,
    schedule_interval="0 12 * * *",
    )

for i in range(10):
    output_one_path = os.path.join(DATA_FOLDER, "output_one_{:d}.txt".format(i))
    output_two_path = os.path.join(DATA_FOLDER, "output_two_{:d}.txt".format(i))

    t1 = BashOperator(
        task_id="print_file_{:d}".format(i),
        bash_command='echo "pipeline" > {}'.format(output_one_path),
        dag=dag)

    t2 = PythonOperator(
        task_id="decorate_file_{:d}".format(i),
        python_callable=decorate_file,
        op_kwargs={"input_path": output_one_path, "output_path": output_two_path},
        dag=dag)

    t2.set_upstream(t1)

<ipython-input-3-9a97bf721742>:6 DeprecationWarning: The `airflow.operators.bash_operator.BashOperator` class is deprecated. Please use `'airflow.operators.bash.BashOperator'`.

<ipython-input-3-9a97bf721742>:7 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

<ipython-input-3-9a97bf721742>:31 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.

# Pipeline 정의

1. 숫자가 담긴 리스트 정의
2. 리스트에 임의의 숫자 추가
3. 리스트의 합계 계산
4. 계산된 합계를 output.txt 파일에 저장

### with DAG(...) as dag: 안에는 기본적인 DAG 수행 조건을 정의해 준다.
- DAG 이름
- 시작 시간
- 반복 조건 (몇 분마다 실행할지, 한 번만 실행할지 등등...)
- 함수 리턴값을 이용하는 경우 context 사용 여부 설정

In [4]:
from airflow import DAG
from airflow.operators.python import PythonOperator
from datetime import datetime, timedelta
import logging


# 1. Get 'num_list' as an argument of function
# 2. Insert 10 to 'num_list'
# 3. Return list
def func_insert_list(num_list):
    num_list.append(10)
    return num_list

# 1. Get num_list (Return value of 'func_get_list')
# 2. Get sum value of the list
# 3. Return sum value
def func_sum_list(**context):
    # Get return value of funtion 'func_insert_list' using task id 'id_insert_number_list'
    response_num_list = context['task_instance'].xcom_pull(
        task_ids='id_insert_number_list')
    logging.info(response_num_list)
    sum_val = sum(response_num_list)
    return sum_val

# Store data to output.txt
def func_save(**context):
    # Get return value of function 'func_sum_list' using task id 'id_sum_list'
    sum_val = context['task_instance'].xcom_pull(task_ids='id_sum_list')
    logging.info(f"Sum of list: {sum_val}")
    with open("output.txt", "w") as file:
        file.write(f"Sum of list: {sum_val}")

#########################################################
# Define DAG

default_args = {
    'owner': 'airflow',
    "start_date": datetime(2023, 1, 1),
    # Receive the context information as kwargs in the function
    "provide_context": True
}

with DAG(
    # DAG name
    'sum_of_list',
    default_args=default_args,
    # Run every 10 minutes
    schedule_interval=timedelta(minutes=10),
) as dag:

    # Define number list
    num_list = [1, 2, 3]

    # Define task
    insert_list = PythonOperator(
        # Define task ID
        task_id='id_insert_number_list',
        # Function to run
        python_callable=func_insert_list,
        # Specify the arguments for the function
        op_args=[num_list],
        dag=dag
    )

    # Define task
    sum_list = PythonOperator(
        # Define task ID
        task_id='id_sum_list',
        # Function to run
        python_callable=func_sum_list,
        dag=dag
    )

    # Define task
    save = PythonOperator(
        # Define task ID
        task_id='id_saving_data',
        # Function to run
        python_callable=func_save,
        dag=dag
    )

    # Task relationship
    insert_list >> sum_list >> save

<ipython-input-4-6e820e789ae7>:43 RemovedInAirflow3Warning: Param `schedule_interval` is deprecated and will be removed in a future release. Please use `schedule` instead.